In [1]:
! pip install --quiet torch torchvision matplotlib pytorch-lightning
! pip install --quiet ipywidgets IProgress
! jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from IPython.display import set_matplotlib_formats
from matplotlib.colors import to_rgba
from tqdm.notebook import tqdm
from torch.nn.modules.conv import Conv2d, ConvTranspose2d
from torch.nn.modules.activation import ReLU, Sigmoid
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
import torchvision
%matplotlib inline

print("Using torch", torch.__version__)
torch.manual_seed(42)
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
NUM_GPUS = len(available_gpus)
print("GPUs avaiable: ", NUM_GPUS)


Using torch 1.11.0a0+17540c5
Is the GPU available? True
Device cuda
GPUs avaiable:  1


In [14]:
BATCH_SIZE = 32

transform = transforms.Compose([
      transforms.ToTensor(),                 
])
lfw_data = datasets.LFWPeople(root="./train_data", split="train", download=True, transform=transform)
lfw_data_val = datasets.LFWPeople(root="./val_data", split="test", download=True, transform=transform)
data_loader = DataLoader(dataset=lfw_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
data_loader_val = DataLoader(dataset=lfw_data_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
class AutoEncoderVGG(pl.LightningModule):
    def __init__(self, block=1, include_feature_loss=False):
        super().__init__()
        self.block = block
        self.include_feature_loss = include_feature_loss
        self.encoder = self.create_encoder()
        self.decoder = self.create_decoder()
    
    def create_encoder(self):
        if self.block == 1:
            vgg19 = torchvision.models.vgg19(pretrained=True)
            vgg_layers = list(list(vgg19.children())[0].children())[:-33]
            vgg = nn.Sequential(*vgg_layers)
            for p in vgg.parameters():
                p.requires_grad = False
            return vgg
        elif self.block == 3:
            vgg19 = torchvision.models.vgg19(pretrained=True)
            vgg_layers = list(list(vgg19.children())[0].children())[:-19]
            vgg = nn.Sequential(*vgg_layers)
            for p in vgg.parameters():
                p.requires_grad = False
            return vgg
        else:
            raise ValueError("Please supply an encoding layer of either 1 or 3")
    
    def create_decoder(self):
        if self.block == 1:
            return nn.Sequential(
                nn.ConvTranspose2d(64, 64, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(64, 3, 3, stride=1, padding=1),
                nn.Sigmoid())
        elif self.block == 3:
            return nn.Sequential(
                nn.ConvTranspose2d(256, 256, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(256, 256, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(256, 256, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(256, 128, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.Upsample(scale_factor=2),
                nn.ReLU(),
                nn.ConvTranspose2d(128, 128, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(128, 64, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.Upsample(scale_factor=2),
                nn.ReLU(),
                nn.ConvTranspose2d(64, 64, 3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(64, 3, 3, stride=1, padding=0),
                nn.Sigmoid())
        else:
            raise ValueError("Please supply an encoding layer of either 1 or 3")

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)
    
    def training_step(self, batch, batch_idx):
        # Autoencode input batch 
        img, labels = batch    
        recon = self.forward(img)
        if self.include_feature_loss:
            # Calculate reconstruction loss
            reconstruction_loss = F.mse_loss(img, recon)
            # Calculate feature loss
            feature_loss = F.mse_loss(self.encode(img), self.encode(recon))
            # Add L2 Norms
            loss = reconstruction_loss + feature_loss
        else:
            # Calculate reconstruction loss
            loss = F.mse_loss(img, recon)

        self.log("train_loss", loss)
        #Lightning trainer handles zero_grad, loss.backwards, and optimizer.step
        return loss
    
    def validation_step(self, batch, batch_idx):
        # Autoencode input batch 
        img, labels = batch    
        recon = self.forward(img)
        # Calculate reconstruction loss
        loss = F.mse_loss(img, recon)
        self.log("val_loss", loss)
        return loss
                                      
    def encode(self, x):
        return self.encoder(x)
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [20]:
model = AutoEncoderVGG(block=1)
logger = CSVLogger("logs", name="vgg_block_1_autoencoder")
checkpoint_callback = ModelCheckpoint(dirpath="block_1_checkpoints/", save_top_k=2, monitor="val_loss")
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", devices=NUM_GPUS, callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=2), checkpoint_callback], logger=logger)
should_train = True
if should_train: trainer.fit(model, train_dataloaders=data_loader, val_dataloaders=data_loader_val)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 38.7 K
1 | decoder | Sequential | 38.7 K
---------------------------------------
38.7 K    Trainable params
38.7 K    Non-trainable params
77.4 K    Total params
0.310     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [21]:
# Lightning automatically checkpoints our model during training
# However to save a seperate checkpoint so we can ensure we do not
# accidentally override we can save manually
should_save = True
if should_save: trainer.save_checkpoint("block_1_model/model.ckpt")
should_load = True
if should_load: model = AutoEncoderVGG.load_from_checkpoint(checkpoint_path="block_1_model/model.ckpt", block=1)

In [15]:
model = AutoEncoderVGG(block=3, include_feature_loss=True)
logger = CSVLogger("logs", name="vgg_block_3_autoencoder")
checkpoint_callback = ModelCheckpoint(dirpath="block_3_checkpoints/", save_top_k=2, monitor="val_loss")
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", devices=NUM_GPUS, callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=2), checkpoint_callback], logger=logger, precision=16)
should_train = True
if should_train: trainer.fit(model, train_dataloaders=data_loader, val_dataloaders=data_loader_val)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 2.3 M 
1 | decoder | Sequential | 2.3 M 
---------------------------------------
2.3 M     Trainable params
2.3 M     Non-trainable params
4.7 M     Total params
9.302     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [12]:
# Lightning automatically checkpoints our model during training
# However to save a seperate checkpoint so we can ensure we do not
# accidentally override we can save manually
should_save = True
if should_save: trainer.save_checkpoint("block_3_model/model.ckpt")
should_load = True
if should_load: model = AutoEncoderVGG.load_from_checkpoint(checkpoint_path="block_3_model/model.ckpt", block=3)

In [9]:
# None of this is used now since we are using lightning
# model = AutoEncoderVGGBlock3()

# # Setup hyperparameters
# # Configure lr and optimizer, may want to add weight decay
# learning_rate = 1e-3
# optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # We calculate the reconstruciton loss using l2 Norm
# recon_loss_func = torch.nn.MSELoss()
# feature_loss_func = torch.nn.MSELoss()

# # Number of epochs with batch size of 48 (may change this) on ~13k images per epoch
# num_epochs = 10

# train_vgg_block_3_ae = False
# block_3_train_outputs = []
# block_3_losses = []

# if train_vgg_block_3_ae == True:
#     # Move the model to GPU if exists for training
#     model = model.to(device)
    
#     for epoch in range(num_epochs):
#         for img, _ in tqdm(data_loader):
#             # Move batch to GPU if available
#             img = img.to(device)
#             # Autoencode input batch
#             recon = model(img)
#             # Calculate reconstruction loss
#             recon_loss = recon_loss_func(img, recon)
#             # Calculate feature loss
#             feature_loss = feature_loss_func(model.encode(img), model.encode(recon))
#             # Calculate combined l2 norm
#             total_loss = recon_loss + feature_loss
#             # Reset optimizer gradients from previous batch
#             optim.zero_grad()
#             # Calculate gradient loss
#             total_loss.backward()
#             # Update parameters based on current gradient
#             optim.step()

#         # If the current epoch is the final epoch then we want to save the reconstructions and the input just so we can
#         # check how things are learning
#         # We should remove this later
#         if epoch == num_epochs - 1: outputs.append((epoch, img.detach().cpu().numpy(), recon.detach().cpu().numpy()))
#         block_3_train_outputs.append((epoch, img.detach().cpu().numpy(), recon.detach().cpu().numpy()))
#         block_3_losses.append(loss.item())
#         print(f'Epoch: {epoch + 1}, Loss:{loss.item():.4f}')
#     # Move the model back to CPU to free up GPU memory
#     model = model.cpu()
    
# else:
#     model.load_state_dict(torch.load("models/vgg_ae_block_3.pth"))
#     block_3_losses = np.loadtxt("models/vgg_ae_block_3_losses.txt").reshape(num_epochs, 1)
#     print("Loaded AutoEncoderVGGBlock3 from disk")

Loaded AutoEncoderVGGBlock3 from disk


### 3 Choices for Style Transfer
- [] Image Masking for multi style transfer
- [] Blending multiple styled images
- [] HSV Color Preserving Style Transfer

### Things for Nick to Do:
- [-] Lightning Module
- [-] Embedding loss for block 1 (use the parameter: include_feature_loss) (I could not train this for long because 
I had to reduce batch size to fit on the GPU, so I do not know how well this works, try increasing the batch size and running on maneframe)
- [-] Git repo
- [-] Display evaluations
- [-] Clean up model save/loading
- [-] Save losses in the training loop